In [1]:
import pandas as pd
import altair as alt
alt.data_transformers.disable_max_rows()

url = "https://raw.githubusercontent.com/UIUC-iSchool-DataViz/is445_data/main/building_inventory.csv"
df = pd.read_csv(url)
df

,Agency Name,Location Name,Address,City,Zip code,County,Congress Dist,Congressional Full Name,Rep Dist,Rep Full Name,...,Bldg Status,Year Acquired,Year Constructed,Square Footage,Total Floors,Floors Above Grade,Floors Below Grade,Usage Description,Usage Description 2,Usage Description 3
0,Department of Natural Resources,Anderson Lake Conservation Area - Fulton County,Anderson Lake C.a.,Astoria,61501,Fulton,17,Cheri Bustos,93,Hammond Norine K.,...,In Use,1975,1975,144,1,1,0,Unusual,Unusual,Not provided
1,Department of Natural Resources,Anderson Lake Conservation Area - Fulton County,Anderson Lake C.a.,Astoria,61501,Fulton,17,Cheri Bustos,93,Hammond Norine K.,...,In Use,2004,2004,144,1,1,0,Unusual,Unusual,Not provided
2,Department of Natural Resources,Anderson Lake Conservation Area - Fulton County,Anderson Lake C.a.,Astoria,61501,Fulton,17,Cheri Bustos,93,Hammond Norine K.,...,In Use,2004,2004,144,1,1,0,Unusual,Unusual,Not provided
3,Department of Natural Resources,Anderson Lake Conservation Area - Fulton County,Anderson Lake C.a.,Astoria,61501,Fulton,17,Cheri Bustos,93,Hammond Norine K.,...,In Use,2004,2004,144,1,1,0,Unusual,Unusual,Not provided
4,Department of Natural Resources,Anderson Lake Conservation Area - Fulton County,Anderson Lake C.a.,Astoria,61501,Fulton,17,Cheri Bustos,93,Hammond Norine K.,...,In Use,2004,2004,144,1,1,0,Unusual,Unusual,Not provided
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8857,Department of Transportation,Belvidere Maintenance Storage Facility - Boone...,9797 Illinois Rte. 76,Belvidere,61008,Boone,16,Adam Kinzinger,69,Sosnowski Joe,...,In Use,0,0,432,1,0,0,Storage,NaN,NaN
8858,Department of Transportation,Belvidere Maintenance Storage Facility - Boone...,9797 Illinois Rte 76,Belvidere,61008,Boone,16,Adam Kinzinger,69,Sosnowski Joe,...,In Use,0,0,330,1,0,0,Storage,NaN,NaN
8859,Department of Transportation,Quincy Maintenance Storage Facility,800 Koch's Lane,Quincy,62305,Adams,18,Darin M. LaHood,94,Frese Randy E.,...,In Use,0,1987,130,1,0,0,Storage,High Hazard,NaN
8860,Illinois Community College Board,Illinois Valley Community College - Oglesby,815 North Orlando Smith Avenue,Oglesby,61348,LaSalle,16,Adam Kinzinger,76,Long Jerry Lee,...,In Use,1971,1971,49552,1,1,0,Education,Education,Not provided


In [2]:
df['County'].fillna('Unknown', inplace=True)
df['Rep Full Name'].fillna('Not Available', inplace=True)
df['Year Constructed'].fillna(df['Year Constructed'].median(), inplace=True)
df['Senator Full Name'].fillna('Unknown', inplace=True)
df['Usage Description 2'].fillna(df['Usage Description 2'].mode()[0], inplace=True)
df['Usage Description 3'].fillna(df['Usage Description 3'].mode()[0], inplace=True)
df['Address'].fillna('Address Not Available', inplace=True)
df['Congressional Full Name'].fillna('Unknown Congressional Name', inplace=True)

/var/folders/ww/7btkbw2n1td4hg84gtnm3s740000gn/T/ipykernel_90269/2550189836.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['County'].fillna('Unknown', inplace=True)
/var/folders/ww/7btkbw2n1td4hg84gtnm3s740000gn/T/ipykernel_90269/2550189836.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always b

In [3]:
county_data = df.groupby(['County', 'Usage Description'])['Square Footage'].sum().reset_index()

usage_description_dropdown = alt.binding_select(options=df['Usage Description'].dropna().unique(), name="Select Usage Description")
usage_description_selection = alt.selection_single(fields=['Usage Description'], bind=usage_description_dropdown, name="usage_description_selection")

county_data_plot = alt.Chart(county_data).mark_bar().encode(
    x=alt.X('County:N', title='County'),
    y=alt.Y('Square Footage:Q', title='Total Square Footage'),
    color=alt.Color('Usage Description:N', legend=alt.Legend(title="Usage Description")),
    tooltip=['County', 'Usage Description', 'Square Footage']
).add_selection(
    usage_description_selection
).transform_filter(
    usage_description_selection
).properties(
    title="Square Footage by County",
    width=800,
    height=600
)
county_data_plot


/var/folders/ww/7btkbw2n1td4hg84gtnm3s740000gn/T/ipykernel_90269/2635961062.py:4: AltairDeprecationWarning: Deprecated in `altair=5.0.0`. Use selection_point instead.
  usage_description_selection = alt.selection_single(fields=['Usage Description'], bind=usage_description_dropdown, name="usage_description_selection")
/var/folders/ww/7btkbw2n1td4hg84gtnm3s740000gn/T/ipykernel_90269/2635961062.py:6: AltairDeprecationWarning: Deprecated in `altair=5.0.0`. Use add_params instead.
  county_data_plot = alt.Chart(county_data).mark_bar().encode(


alt.Chart(...)

In [4]:
agency_data = df.groupby(['Agency Name', 'Congress Dist', 'Bldg Status'])['Square Footage'].sum().reset_index()

bldg_status_dropdown = alt.binding_select(options=df['Bldg Status'].dropna().unique(), name="Select Building Status")
bldg_status_selection = alt.selection_single(fields=['Bldg Status'], bind=bldg_status_dropdown, name="bldg_status_selection")

bubble_chart_building_status = alt.Chart(agency_data).mark_circle(size=100).encode(
    y=alt.Y('Congress Dist:N', title='Congressional District'),
    x=alt.X('Agency Name:N', title='Agency Name', axis=alt.Axis(labelAngle=-45)),
    size=alt.Size('Square Footage:Q', title='Square Footage', scale=alt.Scale(range=[50, 500])), 
    color=alt.Color('Bldg Status:N', legend=alt.Legend(title="Building Status")), 
    tooltip=['Agency Name', 'Congress Dist', 'Square Footage', 'Bldg Status']
).add_selection(
    bldg_status_selection 
).transform_filter(
    bldg_status_selection
).properties(
    title="Bubble Chart of Square Footage by Agency and Congressional District",
    width=800,
    height=600
)
bubble_chart_building_status


/var/folders/ww/7btkbw2n1td4hg84gtnm3s740000gn/T/ipykernel_90269/1782806280.py:4: AltairDeprecationWarning: Deprecated in `altair=5.0.0`. Use selection_point instead.
  bldg_status_selection = alt.selection_single(fields=['Bldg Status'], bind=bldg_status_dropdown, name="bldg_status_selection")
/var/folders/ww/7btkbw2n1td4hg84gtnm3s740000gn/T/ipykernel_90269/1782806280.py:6: AltairDeprecationWarning: Deprecated in `altair=5.0.0`. Use add_params instead.
  bubble_chart_building_status = alt.Chart(agency_data).mark_circle(size=100).encode(


alt.Chart(...)

In [5]:
county_data_plot.save(r'/Users/tanvi/Homwork6.1/assets/visualizations/chart1.html')


bubble_chart_building_status.save(r'/Users/tanvi/Homwork6.1/assets/visualizations/chart2.html')